In [3]:
!pip install python-geohash

  Preparing metadata (setup.py) ... done
  Created wheel for python-geohash: filename=python_geohash-0.8.5-cp311-cp311-linux_x86_64.whl size=41948 sha256=d5c9c1948273367a850810ae5fedc8e3692520c27d4c87b1e9289d9a808d3be4
  Stored in directory: /root/.cache/pip/wheels/02/7a/f4/c27d535af1a4ad8c1112e1bf299d9d47b57fe0fa2a464e4795
Successfully built python-geohash


In [4]:
import pandas as pd
import folium
import geohash
from matplotlib import colors as mcolors

In [10]:
df = pd.read_csv('/kaggle/input/new-new/actual_df_144 (1).csv')  
cell_df = df[df['Cell_ID'] == 'Cell_144'].copy()

sample_size = min(3000, len(cell_df))
cell_df = cell_df.sample(n=sample_size, random_state=42)

cell_df['geohash6'] = cell_df['Geohash'].str[:6]

agg_data = []
for gh, group in cell_df.groupby('geohash6'):
    lat, lon = geohash.decode(gh)  
    avg_rsrp = group['RSRP'].mean()
    count = len(group)
    agg_data.append((gh, lat, lon, avg_rsrp, count))

agg_df = pd.DataFrame(agg_data, columns=['geohash6','avg_lat','avg_lon','avg_RSRP','point_count'])

vmin = agg_df['avg_RSRP'].quantile(0.05)
vmax = agg_df['avg_RSRP'].quantile(0.95)
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = mcolors.LinearSegmentedColormap.from_list("", ["red","orange","yellow","green"])

center_lat = agg_df['avg_lat'].mean()
center_lon = agg_df['avg_lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

for _, row in agg_df.iterrows():
    bbox = geohash.bbox(row['geohash6'])
    
    if row['avg_RSRP'] == 0:
        color = "#606060"  
        popup_text = f"Points: {row['point_count']}<br>Avg RSRP: 0"
    else:
        color = mcolors.to_hex(cmap(norm(row['avg_RSRP'])))
        popup_text = f"Points: {row['point_count']}<br>Avg RSRP: {row['avg_RSRP']:.2f}"

    folium.Rectangle(
        bounds=[[bbox['s'], bbox['w']], [bbox['n'], bbox['e']]],
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(m)

if 'Site_latitude' in cell_df.columns and 'Site_longitude' in cell_df.columns:
    site_lat = cell_df['Site_latitude'].iloc[0]
    site_lon = cell_df['Site_longitude'].iloc[0]

    folium.Marker(
        location=[site_lat, site_lon],
        icon=folium.Icon(color='black', icon='signal', prefix='fa'),
        popup=f"Cell Site: {cell_df['Cell_ID'].iloc[0]}"
    ).add_to(m)

title_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 50%;
    transform: translateX(-50%);
    z-index: 9999;
    font-size: 22px;
    font-weight: bold;
    background-color: white;
    padding: 5px 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
Cell_ID:144 (MDT RSRP Distribution Map - ACTUAL)
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: 140px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 padding:5px;
                 ">
     &nbsp;<b>RSRP (dBm)</b><br>
     &nbsp;<i style="background:red;color:red">....</i>&nbsp; Weak<br>
     &nbsp;<i style="background:orange;color:orange">....</i>&nbsp; Medium-low<br>
     &nbsp;<i style="background:yellow;color:yellow">....</i>&nbsp; Medium-high<br>
     &nbsp;<i style="background:green;color:green">....</i>&nbsp; Strong<br>
     &nbsp;<i style="background:#606060;color:#606060">....</i>&nbsp; Zero RSRP
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

m.save("Cell_144_geohash6_distribution_map_actual.html")
m

In [11]:
df = pd.read_csv('/kaggle/input/new-new/predicted_df_144 (1).csv')  
cell_df = df[df['Cell_ID'] == 'Cell_144'].copy()

sample_size = min(3000, len(cell_df))
cell_df = cell_df.sample(n=sample_size, random_state=42)

cell_df['geohash6'] = cell_df['Geohash'].str[:6]

agg_data = []
for gh, group in cell_df.groupby('geohash6'):
    lat, lon = geohash.decode(gh)   
    avg_rsrp = group['RSRP'].mean()
    count = len(group)
    agg_data.append((gh, lat, lon, avg_rsrp, count))

agg_df = pd.DataFrame(agg_data, columns=['geohash6','avg_lat','avg_lon','avg_RSRP','point_count'])

vmin = agg_df['avg_RSRP'].quantile(0.05)
vmax = agg_df['avg_RSRP'].quantile(0.95)
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
cmap = mcolors.LinearSegmentedColormap.from_list("", ["red","orange","yellow","green"])

center_lat = agg_df['avg_lat'].mean()
center_lon = agg_df['avg_lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

for _, row in agg_df.iterrows():
    bbox = geohash.bbox(row['geohash6'])
    
    if row['avg_RSRP'] == 0:
        color = "#606060"  
        popup_text = f"Points: {row['point_count']}<br>Avg RSRP: 0"
    else:
        color = mcolors.to_hex(cmap(norm(row['avg_RSRP'])))
        popup_text = f"Points: {row['point_count']}<br>Avg RSRP: {row['avg_RSRP']:.2f}"

    folium.Rectangle(
        bounds=[[bbox['s'], bbox['w']], [bbox['n'], bbox['e']]],
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(m)

if 'Site_latitude' in cell_df.columns and 'Site_longitude' in cell_df.columns:
    site_lat = cell_df['Site_latitude'].iloc[0]
    site_lon = cell_df['Site_longitude'].iloc[0]

    folium.Marker(
        location=[site_lat, site_lon],
        icon=folium.Icon(color='black', icon='signal', prefix='fa'),
        popup=f"Cell Site: {cell_df['Cell_ID'].iloc[0]}"
    ).add_to(m)

title_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 50%;
    transform: translateX(-50%);
    z-index: 9999;
    font-size: 22px;
    font-weight: bold;
    background-color: white;
    padding: 5px 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
Cell_ID:144 (MDT RSRP Distribution Map - PREDICTION)
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: 140px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 padding:5px;
                 ">
     &nbsp;<b>RSRP (dBm)</b><br>
     &nbsp;<i style="background:red;color:red">....</i>&nbsp; Weak<br>
     &nbsp;<i style="background:orange;color:orange">....</i>&nbsp; Medium-low<br>
     &nbsp;<i style="background:yellow;color:yellow">....</i>&nbsp; Medium-high<br>
     &nbsp;<i style="background:green;color:green">....</i>&nbsp; Strong<br>
     &nbsp;<i style="background:#606060;color:#606060">....</i>&nbsp; Zero RSRP
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

m.save("Cell_144_geohash6_distribution_map_prediction.html")
m

In [27]:
import pandas as pd
import numpy as np
import folium
import geohash
import matplotlib.colors as mcolors
from scipy.stats import pearsonr

def compare_rsrp_maps_black_missing(pred_csv, actual_csv, cell_id, generate_map=True):
    """
    Compare predicted vs actual RSRP maps for a given Cell_ID, including all geohash6 cells.
    Missing cells are colored black on the difference map.
    
    Args:
        pred_csv (str): Path to predicted CSV.
        actual_csv (str): Path to actual CSV.
        cell_id (str): Cell_ID to filter.
        generate_map (bool): If True, generates folium difference map.
        
    Returns:
        dict: similarity metrics and optionally folium map object.
    """
    # -------------------------
    # Load CSVs
    # -------------------------
    df_pred = pd.read_csv(pred_csv)
    df_actual = pd.read_csv(actual_csv)
    
    df_pred = df_pred[df_pred['Cell_ID'] == cell_id].copy()
    df_actual = df_actual[df_actual['Cell_ID'] == cell_id].copy()
    
    # Sample if necessary (optional)
    sample_size = min(3000, len(df_pred))
    df_pred = df_pred.sample(n=sample_size, random_state=42)
    sample_size = min(3000, len(df_actual))
    df_actual = df_actual.sample(n=sample_size, random_state=42)
    
    # -------------------------
    # Aggregate by geohash6
    # -------------------------
    df_pred['geohash6'] = df_pred['Geohash'].str[:6]
    df_actual['geohash6'] = df_actual['Geohash'].str[:6]
    
    def aggregate(df):
        agg_data = []
        for gh, group in df.groupby('geohash6'):
            lat, lon = geohash.decode(gh)
            avg_rsrp = group['RSRP'].mean()
            count = len(group)
            agg_data.append((gh, lat, lon, avg_rsrp, count))
        return pd.DataFrame(agg_data, columns=['geohash6','avg_lat','avg_lon','avg_RSRP','point_count'])
    
    agg_pred = aggregate(df_pred)
    agg_actual = aggregate(df_actual)
    
    # -------------------------
    # Full outer merge to include all geohash6 cells
    # -------------------------
    merged = pd.merge(
        agg_pred[['geohash6', 'avg_RSRP', 'avg_lat', 'avg_lon']],
        agg_actual[['geohash6', 'avg_RSRP']],
        on='geohash6',
        how='outer',
        suffixes=('_pred', '_actual')
    )
    
    # Fill missing lat/lon from other dataset or decode from geohash
    merged['avg_lat'] = merged['avg_lat'].combine_first(
        merged['geohash6'].apply(lambda gh: geohash.decode(gh)[0])
    )
    merged['avg_lon'] = merged['avg_lon'].combine_first(
        merged['geohash6'].apply(lambda gh: geohash.decode(gh)[1])
    )
    
    # -------------------------
    # Compute similarity metrics only for overlapping cells
    # -------------------------
    overlap = merged.dropna(subset=['avg_RSRP_pred', 'avg_RSRP_actual'])
    corr, _ = pearsonr(overlap['avg_RSRP_pred'], overlap['avg_RSRP_actual'])
    mae = np.mean(np.abs(overlap['avg_RSRP_pred'] - overlap['avg_RSRP_actual']))
    rmse = np.sqrt(np.mean((overlap['avg_RSRP_pred'] - overlap['avg_RSRP_actual'])**2))
    similarity_score = (corr + 1) / 2  # maps [-1,1] -> [0,1]
    
    results = {
        'MAE': mae,
        'RMSE': rmse,
        'Pearson Correlation': corr,
        'Similarity Score (0-1)': similarity_score
    }
    
    # -------------------------
    # Optional: create difference map
    # -------------------------
    if generate_map:
        merged['RSRP_diff'] = merged['avg_RSRP_pred'] - merged['avg_RSRP_actual']
        center_lat = merged['avg_lat'].mean()
        center_lon = merged['avg_lon'].mean()
        
        # Compute color normalization only for overlapping cells
        vmax = merged['RSRP_diff'].abs().quantile(0.95)
        if vmax < 2:
            vmax = 2
        norm_diff = mcolors.TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)
        cmap_diff = mcolors.LinearSegmentedColormap.from_list("", ["blue","white","red"])
        
        m_diff = folium.Map(location=[center_lat, center_lon], zoom_start=15)
        
        for _, row in merged.iterrows():
            bbox = geohash.bbox(row['geohash6'])
            
            if pd.isna(row['avg_RSRP_pred']) or pd.isna(row['avg_RSRP_actual']):
                color = "#000000"  # black for missing cells
                popup_text = "Cell missing in one dataset"
            else:
                color = mcolors.to_hex(cmap_diff(norm_diff(row['RSRP_diff'])))
                popup_text = f"RSRP Diff (Pred - Actual): {row['RSRP_diff']:.2f} dBm"
            
            folium.Rectangle(
                bounds=[[bbox['s'], bbox['w']], [bbox['n'], bbox['e']]],
                color=color, fill=True, fill_opacity=0.6, popup=popup_text
            ).add_to(m_diff)
        
        # Legend with black for missing cells
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; width: 240px;
                    background-color: white; border:2px solid grey; z-index:9999;
                    padding:10px; font-size:14px;">
        <b>RSRP Difference (Pred - Actual)</b><br>
        <i style="background:blue;color:blue">....</i>&nbsp; Under-predicted<br>
        <i style="background:white;color:white;border:1px solid grey">....</i>&nbsp; Accurate<br>
        <i style="background:red;color:red">....</i>&nbsp; Over-predicted<br>
        <i style="background:black;color:black">....</i>&nbsp; Missing cell
        </div>
        '''
        m_diff.get_root().html.add_child(folium.Element(legend_html))
        
        results['Difference Map'] = m_diff
    
    return results


In [28]:
result = compare_rsrp_maps_black_missing(
    pred_csv="/kaggle/input/new-new/predicted_df_144 (1).csv",
    actual_csv="/kaggle/input/new-new/actual_df_144 (1).csv",
    cell_id="Cell_144",
    generate_map=True
)

# Print metrics
for k, v in result.items():
    if k != 'Difference Map':
        print(f"{k}: {v}")

# Show difference map
result['Difference Map']


MAE: 4545.442547196131
RMSE: 6150.98361843343
Pearson Correlation: 0.3103103811044936
Similarity Score (0-1): 0.6551551905522468


In [ ]:
import pandas as pd
import geohash

# Load predicted and actual CSVs
pred_csv = "/kaggle/input/new-new/predicted_df_144 (1).csv"
actual_csv = "/kaggle/input/new-new/actual_df_144 (1).csv"
cell_id = "Cell_144"

df_pred = pd.read_csv(pred_csv)
df_actual = pd.read_csv(actual_csv)

df_pred = df_pred[df_pred['Cell_ID'] == cell_id].copy()
df_actual = df_actual[df_actual['Cell_ID'] == cell_id].copy()

# Aggregate by geohash6
df_pred['geohash6'] = df_pred['Geohash'].str[:6]
df_actual['geohash6'] = df_actual['Geohash'].str[:6]

def aggregate(df):
    agg_data = []
    for gh, group in df.groupby('geohash6'):
        lat, lon = geohash.decode(gh)
        avg_rsrp = group['RSRP'].mean()
        count = len(group)
        agg_data.append((gh, lat, lon, avg_rsrp, count))
    return pd.DataFrame(agg_data, columns=['geohash6','avg_lat','avg_lon','avg_RSRP','point_count'])

agg_pred = aggregate(df_pred)
agg_actual = aggregate(df_actual)

# Full outer merge
merged = pd.merge(
    agg_pred[['geohash6','avg_RSRP','avg_lat','avg_lon']],
    agg_actual[['geohash6','avg_RSRP']],
    on='geohash6',
    how='outer',
    suffixes=('_pred','_actual')
)

# Fill missing lat/lon from geohash
merged['avg_lat'] = merged['avg_lat'].combine_first(merged['geohash6'].apply(lambda gh: geohash.decode(gh)[0]))
merged['avg_lon'] = merged['avg_lon'].combine_first(merged['geohash6'].apply(lambda gh: geohash.decode(gh)[1]))

# Compute absolute error per cell
merged['abs_error'] = (merged['avg_RSRP_pred'] - merged['avg_RSRP_actual']).abs()

# Rank geohash cells by largest error
top_errors = merged.sort_values('abs_error', ascending=False).head(10)  # top 10 worst cells

top_errors[['geohash6','avg_lat','avg_lon','avg_RSRP_pred','avg_RSRP_actual','abs_error']]
